In [1]:
from ase.db import connect
from ase.cluster import Icosahedron
from ase.constraints import FixAtoms
from ase.calculators.emt import EMT
import numpy as np
from nn_optimize import Ensemble_Relaxer
from ase.optimize import BFGS
from matplotlib import pyplot as plt
import os
from ase.calculators.singlepoint import SinglePointCalculator as SPC
from ase.io.trajectory import Trajectory

### Create demo nanoparticles

In [2]:
db = connect('./AuPd-ico-to-relax-10.db')

if db.count() == 0:
    atoms = Icosahedron('Au', noshells=3, latticeconstant=4.0)
    elements = ['Au', 'Pd']
    nrgs = [] 
    configs = []

    while len(nrgs) < 10:
        symbols = np.random.choice(elements, size=len(atoms), replace=True)
        dummy = atoms.copy()
        dummy.center(vacuum=7.0)
        dummy.set_chemical_symbols(symbols)
        dummy.set_calculator(EMT())
        cnst = FixAtoms(range(13))
        dummy.set_constraint(cnst)
        nrg = dummy.get_potential_energy()
        if nrg not in nrgs:
            nrgs.append(nrg)
            configs.append(dummy)

    for atoms in configs:
        atoms.rattle(0.1)

    for config in configs:
        db.write(config)


### NN ensemble relaxer
 - Run NN relaxer like ASE optimzer
 - Need to provide a ASE calculator into the relaxer class, assume the relaxation job runs locally
 - The neural network training and atoms relxation run sequentially, since have no idea about users' job submit system
 - Preset hyperparameters to train NN and atoms relaxation, like learning rate, NN structure, relaxation max step

In [3]:
nn_params = {'layer_nodes': [40, 40], 'activations': ['tanh', 'tanh'], 'lr': 1}
alpha = 2.0  # confidence coeffience, used to control in what extend we trust the NN model

relaxer = Ensemble_Relaxer(db=db, calculator=EMT(), jobname='AuPd-nano-test', ensemble_size=10, alpha=alpha, nn_params=nn_params)

# relaxer.run() returns a ase-db containing relaxed configurations
relaxed_db = relaxer.run(fmax=0.05, steps=50)

Step 0: get groud truth data
Step 0: groud truth data calculation done
max force for each configuration: 
[2.49, 2.719, 2.677, 2.547, 2.321, 2.696, 2.576, 2.439, 2.435, 2.25]
Step 0: start training


KeyboardInterrupt: 

In [ ]:
nn_max_fs = []
for entry in relaxed_db.select():
    atoms = entry.toatoms()
    max_f = np.linalg.norm(atoms.get_forces(), axis=1).max()
    nn_max_fs.append(round(max_f, 3))
print(nn_max_fs)

### BFGS

In [ ]:

if not os.path.isdir('AuPd-nano-test/AuPd-nano-BFGS'):
    os.mkdir('AuPd-nano-test/AuPd-nano-BFGS')

c = 0
for entry in db.select():
    atoms = entry.toatoms()
    atoms.set_calculator(EMT())
    c += 1
    dyn = BFGS(atoms, trajectory=f'AuPd-nano-test/AuPd-nano-BFGS/config-{c}.traj')
    dyn.run(0.05, steps=50)

In [ ]:
def retrive_nn_trajs(init_db, n_steps, fmax):
    init_cnstrt = [entry.toatoms().constraints for entry in init_db.select()]
    config_steps = [[] for _ in range(init_db.count())]
    config_convgs = [False] * init_db.count()

    for i in range(n_steps+1):
        tmp_db = connect(os.path.join('AuPd-nano-test/relax_trajs', f'to-cal-step{i}-gt.db'))
        tmp_configs = [entry.toatoms() for entry in tmp_db.select()]
        tmp_frs = [atoms.get_forces() for atoms in tmp_configs]
        tmp_nrgs = [atoms.get_potential_energy() for atoms in tmp_configs]
        j = 0
        for k, convg in enumerate(config_convgs):
            if not convg:
                f = tmp_frs[j]
                for c in init_cnstrt[k]:
                    c.adjust_forces(None, f)
                mf = np.linalg.norm(f, axis=1).max()
                if mf <= fmax:
                    config_convgs[k] = True
                dummy = tmp_configs[j].copy()
                dummy.set_constraint(init_cnstrt[k])
                dummy.set_calculator(SPC(dummy, energy=tmp_nrgs[j], forces=tmp_frs[j]))
                config_steps[k].append(dummy)
                j += 1

    return config_steps


def retrive_bfgs_trajs():
    config_steps = []
    for i in range(1, 11):
        traj = Trajectory(f'AuPd-nano-test/AuPd-nano-BFGS/config-{i}.traj')
        config_steps.append(list(traj))
    return config_steps

In [ ]:
nn_relax_trajs = retrive_nn_trajs(db, 3, 0.05)
bfgs_relax_trajs = retrive_bfgs_trajs()

In [ ]:
nn_max_frs = []
bfgs_max_frs = []
for traj in nn_relax_trajs:
    config_max_frs = [np.linalg.norm(atoms.get_forces(), axis=1).max() for atoms in traj]
    nn_max_frs.append(config_max_frs)

for traj in bfgs_relax_trajs:
    config_max_frs = [np.linalg.norm(atoms.get_forces(), axis=1).max() for atoms in traj]
    bfgs_max_frs.append(config_max_frs)

fig = plt.figure()
ax = fig.add_subplot(111)
for max_frs in nn_max_frs:
    s1 = ax.scatter(np.arange(len(max_frs)) + 1, max_frs, c='C0', s=7)
for max_frs in bfgs_max_frs:
    s2 = ax.scatter(np.arange(len(max_frs)) + 1, max_frs, c='C1', s=7)
ax.legend([s1, s2], ['NN Ensemble', 'BFGS'])
ax.set_xlabel('EMT Calls')
ax.set_ylabel(r'Max force ($eV/\AA$)')